In [60]:
!pip3 install openai=1.1

Defaulting to user installation because normal site-packages is not writeable
  Using cached pip-24.0-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
import pandas as pd
import numpy as np
import openai
from ast import literal_eval
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from openai import OpenAI
import tiktoken
from scipy import spatial

In [26]:
datafile_path = "datasets/final_dataset.csv"
df = pd.read_csv(datafile_path)
df["embedding"] = df.embedding.apply(literal_eval).apply(np.array)
df["special_embedding"] = df.special_embedding.apply(literal_eval).apply(np.array)

In [15]:
# datafile_path = "datasets/updated_reviews_with_embeddings.csv"
# df2 = pd.read_csv(datafile_path)
# df2["special_embedding"] = df2.special_embedding.apply(literal_eval).apply(np.array)
# df["special_embedding"] = df2["special_embedding"]

In [27]:
df

,Unnamed: 0,Review,Name,combined,n_tokens,special_words,special_embedding,embedding
0,0,"There I was, walking down summer street with my wonderful girlfriend emma. On our way to uno dos go, we were met by an unpleasant surprise. It was closed. Instead of going to our usual burger place, five guys, we decided to head for pizza. Instead of going to the North End we decided to stop into the 1000 degrees pizza shop. Upon our arrival, we were met with heavenly glowing orange walls, lack of music and the sound of an industrial ovens hum and dateline NBC coming from 2 different televisions right next to each other. We were met by a pleasantly harsh woman. Very kind, but her manner was rough. No problem, manners is all I ask for as a patron. We got the ""white out"" and two drinks. However over priced compared to higher quality pizzas in the north end, it was not upset. Then it happened... A strange woman and her son sitting in the booth next to us were preparing to leave. I kept making odd eye contact with the son. Who had a full beard and some of his coca cola would remain by his chin every time he took a sip. Their body language was utterly disturbing. The sad look in their eyes was upsetting and their shy manner was skin crawling. As people normally do, the older woman set sail for the bathroom. This is when things went awry. When she entered the bathroom, who's door was already wide open, the toilet and sink were met with a scream. Not fearful. Not in defense. But a war cry to set dominance over her surroundings. She closed the door and the son stood guard with his coca cola dripping like fresh blood from his bearded face. The screams were like none other. It sounded as though the fear of God was bestowed upon her at that toilet. Clouds rolled over head outside and it seemed as though it got suddenly dark. As the banging on the wall ensued, everyone remained casual and decided to ignore the impending doom. She exited the bathroom. Gathered her belongings. And the dark knight that was her son who defended her valiantly left the building. Once again, we were met with the harsh hum of an industrial oven and dateline nbc",1000 Degrees Pizza Salad Wings,"Name: 1000 Degrees Pizza Salad Wings; Review: There I was, walking down summer street with my wonderful girlfriend emma. On our way to uno dos go, we were met by an unpleasant surprise. It was closed. Instead of going to our usual burger place, five guys, we decided to head for pizza. Instead of going to the North End we decided to stop into the 1000 degrees pizza shop. Upon our arrival, we were met with heavenly glowing orange walls, lack of music and the sound of an industrial ovens hum and dateline NBC coming from 2 different televisions right next to each other. We were met by a pleasantly harsh woman. Very kind, but her manner was rough. No problem, manners is all I ask for as a patron. We got the ""white out"" and two drinks. However over priced compared to higher quality pizzas in the north end, it was not upset. Then it happened... A strange woman and her son sitting in the booth next to us were preparing to leave. I kept making odd eye contact with the son. Who had a full beard and some of his coca cola would remain by his chin every time he took a sip. Their body language was utterly disturbing. The sad look in their eyes was upsetting and their shy manner was skin crawling. As people normally do, the older woman set sail for the bathroom. This is when things went awry. When she entered the bathroom, who's door was already wide open, the toilet and sink were met with a scream. Not fearful. Not in defense. But a war cry to set dominance over her surroundings. She closed the door and the son stood guard with his coca cola dripping like fresh blood from his bearded face. The screams were like none other. It sounded as though the fear of God was bestowed upon her at that toilet. Clouds rolled over head outside and it seemed as though it got suddenly dark. As the banging on the w

## Approach A: Searching reviews using TF-IDF

In [28]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Review'])

def get_reviews_by_tf_idf(query, tfidf_matrix = tfidf_matrix, tfidf_vectorizer = tfidf_vectorizer, df = df):
    query_vector = tfidf_vectorizer.transform([query])
    cosine_similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    top_indices = cosine_similarities.argsort()[-6:-1][::-1]
    return df.iloc[top_indices]['combined'].str.cat(sep='\n') 

## Approach B: Searching reviews using embeddings

In [29]:
client = OpenAI(api_key="") # CONFIDENTIAL, DON'T SHARE
model = "text-embedding-3-small"

def get_embedding(text, model=model):
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding
    
def get_reviews_by_embeddings(query, df = df):
    query_embedding = get_embedding(query)
    similarities = cosine_similarity([query_embedding], df['embedding'].tolist())[0]
    top_indices = similarities.argsort()[-6:-1][::-1] 
    return df.iloc[top_indices]['combined'].str.cat(sep='\n') 

## Approach C: Searching reviews using embeddings amplified by TF-IDF scores

In [30]:
def get_reviews_by_embeddings_tfidf(query, tfidf_matrix=tfidf_matrix, df=df):
    query_embedding = get_embedding(query)
    query_vector = tfidf_vectorizer.transform([query])
    cosine_similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    weighted_embeddings = []
    for index, row in df.iterrows():
        weighted_embedding = row['embedding'] * cosine_similarities[index]
        weighted_embeddings.append(weighted_embedding)
    similarities = cosine_similarity([query_embedding], weighted_embeddings)[0]
    top_indices = similarities.argsort()[-6:-1][::-1]
    return df.iloc[top_indices]['combined'].str.cat(sep='\n\n') 

## Approach D: Searching reviews using embeddings of words within highest 10% TF-IDF scores

In [31]:
def get_reviews_by_embeddings_special_words(query, df=df):
    query_embedding = get_embedding(query)
    similarities = cosine_similarity([query_embedding], df['special_embedding'].tolist())[0]
    top_indices = similarities.argsort()[-6:-1][::-1]
    return df.iloc[top_indices]['combined'].str.cat(sep='\n\n') 

## Testing: Running queries for each approach and determining the best one

In [ ]:
!pip install pdfplumber

In [32]:
import pdfplumber

def extract_queries_from_pdf(pdf_file):
    queries = []
    with pdfplumber.open(pdf_file) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            queries.extend(text.strip().split('\n'))
    return queries

queries = extract_queries_from_pdf('Where2Eat_queries.pdf')
# print(queries)

In [33]:
pd.set_option('display.max_colwidth', None) # so that you are able to see the full columns

In [34]:
test_results = pd.DataFrame()
test_results["Query"] = queries

results_A = []
results_B = []
results_C = []
results_D = []

for query in queries:
    results_A.append(get_reviews_by_tf_idf(query))
    results_B.append(get_reviews_by_embeddings(query))
    results_C.append(get_reviews_by_embeddings_tfidf(query))
    results_D.append(get_reviews_by_embeddings_special_words(query))


test_results["Approach A"] = results_A
test_results["Approach B"] = results_B
test_results["Approach C"] = results_C
test_results["Approach D"] = results_D

In [24]:
test_results

Query  \
0                             I want good appetizers   
1                   Where can I find good cocktails?   
2                         Italian restaurants nearby   
3                            Where can I find sushi?   
4                           Mexican food around here   
..                                               ...   
105               Restaurants for anniversary dinner   
106  Where should I eat with a big group of friends?   
107         Give me some family-friendly restaurants   
108                                    No spicy food   
109                          I don’t like spicy food   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [89]:
# Export DataFrame to a CSV file
test_results.to_csv('results.csv', index=False)  # Set index=False to exclude row indices

## Appendix: Chatbot

In [7]:
GPT_MODEL = 'gpt-3.5-turbo'

def strings_ranked_by_relatedness(query, df, relatedness_fn=lambda x, y: 1 - spatial.distance.cosine(x, y), top_n = 5):
    query_embedding = get_embedding(query)
    strings_and_relatednesses = [
        (row["combined"], relatedness_fn(query_embedding, row["embedding"]))
        for i, row in df.iterrows()
    ]
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)
    strings, relatednesses = zip(*strings_and_relatednesses)
    return strings[:top_n], relatednesses[:top_n]
    
def num_tokens(text, model = GPT_MODEL) -> int:
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

def query_message(query, df, model, token_budget):
    strings, relatednesses = strings_ranked_by_relatedness(query, df)
    introduction = 'Let the user know (1) that the following are the top 3 restaurants recommended to them based on what they have asked for, (2) why those restaurants are a good fit for them and (3) back your justifications with quotes from the reviews. Write in paragraph, conversational style. If the answer cannot be found in the reviews, write "I could not find an answer."'
    question = f"\n\What they asked for: {query}"
    message = introduction
    for string in strings:
        next_article = f'\n\n{string}\n'
        if (num_tokens(message + next_article + question, model=model) > token_budget):
            break
        else:
            message += next_article
    return message + question

def ask(query, df = df, model = GPT_MODEL, token_budget = 4096 - 500, print_message = False):
    message = query_message(query, df, model=model, token_budget=token_budget)
    if print_message:
        print(message)
    messages = [
        {"role": "system", "content": "You answer questions."},
        {"role": "user", "content": message},
    ]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    response_message = response.choices[0].message.content
    return response_message

## Validation

In [8]:
pd.set_option('display.max_colwidth', None) # so that you are able to see the full columns

user_input = "best arab food ever"

In [9]:
get_reviews_by_tf_idf(user_input)

,Name,Review
5034,Mofongo House Restaurant,"Best place ever !! The best place without a doubt. The best Dominican food and the best atmosphere. Excellent!!! If you want to feel at home, you have to visit this place 👌🏽👌🏽👌🏽👌🏽 Love it 😍"
4050,L&C Spanish Restaurant,"Best spanish food, they give great portions the best yet, & not expensive at all, i go there every day & will continue to go"
3230,Giorgiana's,Giorgiana's is honestly the best place in the South End. I recently moved into the city and have had the best experiences here. I love the food and the workers. They always greet me with a smile and are efficient. The greek salad with grilled chicken is a personal favorite. The Obama is one of the best sandwiches. I have talked about this restaurant to other friends who live in the area and they 100% agree with me. BEST food and best selection of wines and other groceries. The owner is great and passionate about her work. Her employees are so friendly and welcoming. I will continue to come here and spread the word about the best place for food in Boston
8257,Topacio,"One of the best if not the best place to have a quick fight or dinner! Love their passion and their food! The pupusas are the best! It made me feel like if I was back home! .. I'm not sure if this place is 100% salvadoran, but what I know is that the people that make the food are! Highly recommend! One of the best, if not the best place to have lunch or dinner! I love their passion and their food! Pupusas are the best! It made me feel like I was at home! .. I'm not sure if this place is 100% Salvadoran, but what I know is that the people who make the food are! I recommend it 👍"
6657,Sate Grill-on-Wheels,BEST FOOD TRUCK IN BOSTON! :


In [10]:
get_reviews_by_embeddings(user_input)

,Name,Review
8478,Uptowne Mediterranean Restaurant,"Best chicken shawarma I've ever had, hands-down. I like it as a fatoush add-on, with a generous schmear of the most amazing garlic sauce, making a simultaneously healthy and decadent meal. The chicken kabobs (both spiced ground meat and marinated chunk versions) are also great. For vegetarian options, the falafel (freshly fried-to-order, not microwaved) and pumpkin kibbeh are fantastic. Honestly, pretty much everything on the menu is really good; if I weren't so addicted to the shawarma fatoush, I'd order other items more often. For those with a sweet tooth, they bake fresh trays daily of baklava and kanafeh. Haven't tried the baklava, but the kanafeh (sort of a Lebanese cheesecake) is a delightful treat. Wonderful, hard-working Lebanese family runs this establishment. Can taste the home-cooked love that gets put into every dish. Big props to them for continuing to grind through this pandemic. Miss being able to enjoy meals in their dining room, but even in takeout the quality of the food remains as sky-high as before"
6811,Shawarma Falafel,"It was a somber Thursday in July when I first visited Shawarma Falafel. Owner Serge gave an earnest smile and said, ""I hope you really enjoy it,"" as I received my chicken shawarma roll-up. And, did I ever enjoy it! The chicken here is amazing—perfectly juicy with just the right level of char and flavor. On a journey to the streets of Beirut, my mouth was in a kaleidoscope of flavors, from the incredible garlic sauce, to the crunchy fries, to the assorted other fillings. I have since experienced the same passion poured into every dish. I can recommend the shawarma, soujouk, kafta, samke harra, and of course, the Falafel. The hummus has just the right blend of seasonings, flavorful yet not overwhelming. Try a platter with tabbouli or fattouch. They're so fresh with a satisfying crunch and seasonings that tantalize. With food so good, and such a lovely experience with the kind owner, I wish I could afford to give Serge my money every day for lunch. I would not tire of the food"
573,BOSTON KABOB COMPANY,"The food in this place is the best mediterranean food I had in Boston area, and is better than many places in Israel"
574,BOSTON KABOB COMPANY,"The food in this place is the best mediterranean food I had in Boston area, and is better than many places in Israel"
7303,Sultan's Kitchen,"I would eat here everyday if I could. Variety abounds, from intensely flavored cold vegetarian salads to the meaty kebabs and specials. If you are lucky enough to hit an eggplant or stuffed pepper or lamb special, go for it without hesitation. The eggplant dishes here are buttery, unctuous, rich with flavor and texture. I haven't experienced a better eggplant treatment anywhere. On a frigid arctic day the soups are steaming hot and stews rich and satisfying. On a Spring or Summer day, get a Kofte or Donner Kebab sandwich wrapped in pita and filled with broiling hot meats and cool Turkish salad with the pita barely holding it all together. There is also this great walk-around pita ""sandwich"" which is great as well"


In [11]:
get_reviews_by_embeddings_tfidf(user_input)

,Name,Review
8478,Uptowne Mediterranean Restaurant,"Best chicken shawarma I've ever had, hands-down. I like it as a fatoush add-on, with a generous schmear of the most amazing garlic sauce, making a simultaneously healthy and decadent meal. The chicken kabobs (both spiced ground meat and marinated chunk versions) are also great. For vegetarian options, the falafel (freshly fried-to-order, not microwaved) and pumpkin kibbeh are fantastic. Honestly, pretty much everything on the menu is really good; if I weren't so addicted to the shawarma fatoush, I'd order other items more often. For those with a sweet tooth, they bake fresh trays daily of baklava and kanafeh. Haven't tried the baklava, but the kanafeh (sort of a Lebanese cheesecake) is a delightful treat. Wonderful, hard-working Lebanese family runs this establishment. Can taste the home-cooked love that gets put into every dish. Big props to them for continuing to grind through this pandemic. Miss being able to enjoy meals in their dining room, but even in takeout the quality of the food remains as sky-high as before"
6811,Shawarma Falafel,"It was a somber Thursday in July when I first visited Shawarma Falafel. Owner Serge gave an earnest smile and said, ""I hope you really enjoy it,"" as I received my chicken shawarma roll-up. And, did I ever enjoy it! The chicken here is amazing—perfectly juicy with just the right level of char and flavor. On a journey to the streets of Beirut, my mouth was in a kaleidoscope of flavors, from the incredible garlic sauce, to the crunchy fries, to the assorted other fillings. I have since experienced the same passion poured into every dish. I can recommend the shawarma, soujouk, kafta, samke harra, and of course, the Falafel. The hummus has just the right blend of seasonings, flavorful yet not overwhelming. Try a platter with tabbouli or fattouch. They're so fresh with a satisfying crunch and seasonings that tantalize. With food so good, and such a lovely experience with the kind owner, I wish I could afford to give Serge my money every day for lunch. I would not tire of the food"
573,BOSTON KABOB COMPANY,"The food in this place is the best mediterranean food I had in Boston area, and is better than many places in Israel"
574,BOSTON KABOB COMPANY,"The food in this place is the best mediterranean food I had in Boston area, and is better than many places in Israel"
8845,Zest Mediterranean Grille,"Lovely restaurant and delicious food I talked with a very respectable Moroccan man working in the restaurant and his wife as well or one of his relatives and the next day I met an Egyptian worker in the restaurant was very nice I ordered the chicken shawarma sandwiches and they were delicious irresistible as well as falafel with tahini My friend asked them rice with shawarma and liked the meal because it was with pickles and they also had baklava and basbousa and a long menu but I did not try it. I will return again, God willing"


In [12]:
print(ask(user_input))

Based on your request for the best Arab food ever, I have found the top 3 restaurants that I believe would be a perfect fit for you. Firstly, Boston Shawarma is highly recommended for its incredible Arab fare, with one reviewer mentioning, "The food is to DIE for." The generous portions, affordable prices, and authentic atmosphere make it a must-visit spot for anyone craving delicious Arab cuisine. Secondly, Uptowne Mediterranean Restaurant stands out for its mouth-watering chicken shawarma and a variety of other delectable dishes. A reviewer raves, "Can taste the home-cooked love that gets put into every dish," highlighting the dedication of the Lebanese family running the establishment. Lastly, Shawarma Falafel offers a delightful culinary experience with dishes like chicken shawarma roll-up and perfectly seasoned hummus. A satisfied customer expressed, "With food so good, and such a lovely experience with the kind owner, I wish I could afford to give Serge my money every day for lun